In [61]:
import ReportTimerange as rtr
import ReportMailer as rm
import GuestbookDbConnect as gdb

In [62]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlsxwriter

In [63]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

In [ ]:
def exit(): raise StopExecution

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days
* SI  - since inception (5/2018)




In [64]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90", "SI"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days", "all dates"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

In [65]:
_INTERVAL = "SI"

In [66]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [67]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [68]:
trange = rtr.timerange(_INTERVAL)
print(trange)

collections = []
summaries = []

collection = {}
collection["name"] = "Outline"

df = pd.DataFrame({"Report Tab":[ \
              "Clients By Total VisitsSummary"], \
              "Description":[ \
              "clients sorted by total visits"]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)



df.head(1000)

[datetime.datetime(2018, 1, 1, 0, 0), datetime.datetime(2022, 1, 7, 16, 44, 47, 62637), 'YYYY-MM']


,Report Tab,Description
0,Clients By Total VisitsSummary,clients sorted by total visits


## Clients by Total Visits in report period ##

In [69]:
summary = {}
summary["name"] = "Clients by Total Visits in report period"

query = "SELECT person.idperson, person.firstname, person.lastname, COUNT(DISTINCT snapshot.idsnapshot) AS clientvisits FROM guestbook_person person \
         JOIN guestbook_personsnapshot snapshot ON person.idperson=snapshot.person_id  \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY person.idperson \
         ORDER BY clientvisits desc ".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

collection["dataframe"] = data
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)

#summary["count"] = data.iloc[0,2]
#summaries.append(summary)
#print(summaries)

data.head(100)

SELECT person.idperson, person.firstname, person.lastname, COUNT(DISTINCT snapshot.idsnapshot) AS clientvisits FROM guestbook_person person          JOIN guestbook_personsnapshot snapshot ON person.idperson=snapshot.person_id           WHERE timestamp BETWEEN '2018-01-01 00:00:00' AND '2022-01-07 16:44:47.062637'          GROUP BY person.idperson          ORDER BY clientvisits desc 


,idperson,firstname,lastname,clientvisits
0,24,CHRIS,MCKINNON,356
1,169,LARRITT,SHEPPARD,335
2,479,REGINALD,MOODY,295
3,104,JERROD,ALLISON,283
4,587,ANTONIO,MORRIS,278
5,473,MARGARET,KEE,275
6,77,STEVE,SLOAN,274
7,41,BILLY,BARNHARDT,259
8,250,STEPHANIE,HELMS,246
9,1038,AARON,AYERS,240


# Summary Counts #

In [70]:
collection = {}
summ = pd.DataFrame.from_dict(summaries)
collection["name"] = "Summary"
collection["dataframe"] = summ
collection["colwidths"] = [30, 60]
collections.insert(1, collection)
#collections.append(collection)
summ.head(1000)

""


In [71]:
_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Reports for {}".format(toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'jahood1@yahoo.com']
_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'pastor@opphouse.net']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

Writing dataframe to Excel file SI-2022-01-07_164525.xlsx
Writing Outline to spreadsheet
colwidths=[30, 30, 30, 30]
Writing Summary to spreadsheet
colwidths=[30, 60]
Writing Outline to spreadsheet
colwidths=[30, 30, 30, 30]
Emailer Done!
Emailer Done!
Excel/Email Done!
